In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [2]:
from google.cloud import bigquery
from google.oauth2 import service_account

In [3]:
key_path = '../service_account/gentle-keyword-423715-j0-03be08ad6412.json'

credentials = service_account.Credentials.from_service_account_file(
    key_path,
    scopes=["https://www.googleapis.com/auth/bigquery"]
)

In [4]:
from google.cloud import bigquery

client = bigquery.Client(
    credentials = credentials,
    project=credentials.project_id
)

In [5]:
# Create dataset reference
dataset_ref = client.dataset('google_analytics_sample', project=credentials.project_id)
# Retrieve dataset from reference
dataset = client.get_dataset(dataset_ref)

In [6]:
# Try if this connected

query = """
SELECT *
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_20160801`
LIMIT 5
"""

df = client.query(query).to_dataframe()
print(df.info())
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   visitorId             0 non-null      Int64 
 1   visitNumber           5 non-null      Int64 
 2   visitId               5 non-null      Int64 
 3   visitStartTime        5 non-null      Int64 
 4   date                  5 non-null      object
 5   totals                5 non-null      object
 6   trafficSource         5 non-null      object
 7   device                5 non-null      object
 8   geoNetwork            5 non-null      object
 9   customDimensions      5 non-null      object
 10  hits                  5 non-null      object
 11  fullVisitorId         5 non-null      object
 12  userId                0 non-null      object
 13  channelGrouping       5 non-null      object
 14  socialEngagementType  5 non-null      object
dtypes: Int64(4), object(11)
memory usage: 752.0+

,visitorId,visitNumber,visitId,visitStartTime,date,totals,trafficSource,device,geoNetwork,customDimensions,hits,fullVisitorId,userId,channelGrouping,socialEngagementType
0,<NA>,1,1470046245,1470046245,20160801,"{'visits': 1, 'hits': 24, 'pageviews': 17, 'ti...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Firefox', 'browserVersion': 'not ...","{'continent': 'Europe', 'subContinent': 'Weste...","[{'index': 4, 'value': 'EMEA'}]","[{'hitNumber': 1, 'time': 0, 'hour': 3, 'minut...",895954260133011192,None,Organic Search,Not Socially Engaged
1,<NA>,1,1470084717,1470084717,20160801,"{'visits': 1, 'hits': 24, 'pageviews': 18, 'ti...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Internet Explorer', 'browserVersi...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 13, 'minu...",0288478011259077136,None,Direct,Not Socially Engaged
2,<NA>,3,1470078988,1470078988,20160801,"{'visits': 1, 'hits': 27, 'pageviews': 17, 'ti...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Safari', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 12, 'minu...",6440789996634275026,None,Organic Search,Not Socially Engaged
3,<NA>,4,1470075581,1470075581,20160801,"{'visits': 1, 'hits': 27, 'pageviews': 19, 'ti...","{'referralPath': '/', 'campaign': '(not set)',...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 11, 'minu...",8520115029387302083,None,Referral,Not Socially Engaged
4,<NA>,30,1470099026,1470099026,20160801,"{'visits': 1, 'hits': 27, 'pageviews': 17, 'ti...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 17, 'minu...",6792260745822342947,None,Organic Search,Not Socially Engaged


In [15]:
query = """
SELECT
  products.v2ProductCategory as category,
  date,
  totals.hits as hits,
  totals.visits as visits,
  totals.pageviews as page_views,
  totals.newVisits as first_visits,
  totals.timeOnScreen as time_on_screen,
  totals.timeOnSite as time_on_site,
  totals.screenviews as screen_views,
  totals.transactions as transactions,
  totals.uniqueScreenviews as unique_screen_views
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*` h,
    UNNEST(h.hits) hits,
    UNNEST(hits.product) products
WHERE 
  (_TABLE_SUFFIX BETWEEN '20160801' AND '20170801')
  AND
    (
      hits.eCommerceAction.action_type != '0'
      AND
      hits.eCommerceAction.action_type != '3'
      AND
      hits.eCommerceAction.action_type != '4'
    )
  AND
  (geoNetwork.country = 'United States')
  AND
  (
  products.v2ProductCategory LIKE '%Accessories%'
  OR products.v2ProductCategory LIKE '%Electronics%'
  OR products.v2ProductCategory LIKE '%Office%'
  OR products.v2ProductCategory LIKE "%Men's%"
  OR products.v2ProductCategory LIKE "%Women's%"
  OR products.v2ProductCategory LIKE "%Kid's%"
  OR products.v2ProductCategory LIKE "%Apparel%"
  OR products.v2ProductCategory LIKE '%Shop by Brand%'
  )
"""

df_totals = client.query(query).to_dataframe()
print(df_totals.info())
df_totals.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 523380 entries, 0 to 523379
Data columns (total 12 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   category             523380 non-null  object
 1   date                 523380 non-null  object
 2   hits                 523380 non-null  Int64 
 3   visits               523380 non-null  Int64 
 4   page_views           523379 non-null  Int64 
 5   first_visits         267367 non-null  Int64 
 6   time_on_screen       0 non-null       Int64 
 7   time_on_site         522951 non-null  Int64 
 8   screen_views         0 non-null       Int64 
 9   transactions         187752 non-null  Int64 
 10  revenue              187446 non-null  Int64 
 11  unique_screen_views  0 non-null       Int64 
dtypes: Int64(10), object(2)
memory usage: 52.9+ MB
None


,category,date,hits,visits,page_views,first_visits,time_on_screen,time_on_site,screen_views,transactions,revenue,unique_screen_views
0,Home/Apparel/Headgear/,20170205,3,1,2,1,<NA>,78,<NA>,<NA>,<NA>,<NA>
1,Home/Apparel/Headgear/,20170205,3,1,2,1,<NA>,78,<NA>,<NA>,<NA>,<NA>
2,Home/Apparel/Men's/Men's-Outerwear/,20170205,3,1,2,1,<NA>,38,<NA>,<NA>,<NA>,<NA>
3,Home/Apparel/Men's/Men's-Outerwear/,20170205,3,1,2,1,<NA>,38,<NA>,<NA>,<NA>,<NA>
4,Home/Apparel/Men's/Men's-T-Shirts/,20170205,4,1,2,<NA>,<NA>,4,<NA>,<NA>,<NA>,<NA>


In [16]:
df_totals = df_totals.dropna(axis=1, how='all').fillna(0)
df_totals['date'] = pd.to_datetime(df_totals['date'])

print(df_totals.info())
df_totals.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 523380 entries, 0 to 523379
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   category      523380 non-null  object        
 1   date          523380 non-null  datetime64[ns]
 2   hits          523380 non-null  Int64         
 3   visits        523380 non-null  Int64         
 4   page_views    523380 non-null  Int64         
 5   first_visits  523380 non-null  Int64         
 6   time_on_site  523380 non-null  Int64         
 7   transactions  523380 non-null  Int64         
 8   revenue       523380 non-null  Int64         
dtypes: Int64(7), datetime64[ns](1), object(1)
memory usage: 39.4+ MB
None


,category,date,hits,visits,page_views,first_visits,time_on_site,transactions,revenue
0,Home/Apparel/Headgear/,2017-02-05,3,1,2,1,78,0,0
1,Home/Apparel/Headgear/,2017-02-05,3,1,2,1,78,0,0
2,Home/Apparel/Men's/Men's-Outerwear/,2017-02-05,3,1,2,1,38,0,0
3,Home/Apparel/Men's/Men's-Outerwear/,2017-02-05,3,1,2,1,38,0,0
4,Home/Apparel/Men's/Men's-T-Shirts/,2017-02-05,4,1,2,0,4,0,0


In [17]:
map_agg = {
    'hits': ['sum', 'mean'],
    'visits': ['sum', 'mean'],
    'page_views': ['sum', 'mean'],
    'first_visits': ['sum', 'mean'],
    'time_on_site': ['sum', 'mean'],
    'transactions': ['sum']    
}

In [18]:
def categorized(x):
    
    def grouping_apparel(x):
        if x in ['Men\'s', 'Women\'s', 'Kid\'s', 'Apparel']:
            return 'Apparel'
        return x
    
    category = []
    categories = ['Accessories', 'Electronics', 'Office', 'Men\'s', 'Women\'s', 'Kid\'s', 'Apparel', 'Shop by Brand']
    for i in x:
        for cat in categories:
            if i.find(cat) != -1:
                category.append(cat)
    if len(category) == 0:
        return None
    
    result = grouping_apparel(category[-1])
    return result


df_totals['category'] = df_totals['category'].str.split('/').apply(categorized)
df_totals = df_totals.groupby(['category', 'date']).agg(map_agg).reset_index()
print(df_totals.info())
df_totals.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1805 entries, 0 to 1804
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   (category, )          1805 non-null   object        
 1   (date, )              1805 non-null   datetime64[ns]
 2   (hits, sum)           1805 non-null   Int64         
 3   (hits, mean)          1805 non-null   Float64       
 4   (visits, sum)         1805 non-null   Int64         
 5   (visits, mean)        1805 non-null   Float64       
 6   (page_views, sum)     1805 non-null   Int64         
 7   (page_views, mean)    1805 non-null   Float64       
 8   (first_visits, sum)   1805 non-null   Int64         
 9   (first_visits, mean)  1805 non-null   Float64       
 10  (time_on_site, sum)   1805 non-null   Int64         
 11  (time_on_site, mean)  1805 non-null   Float64       
 12  (transactions, sum)   1805 non-null   Int64         
dtypes: Float64(5), Int

category       date   hits            visits      page_views             \
                             sum       mean    sum mean        sum       mean   
0  Accessories 2016-08-01  13699  51.307116    267  1.0       9083  34.018727   
1  Accessories 2016-08-02  16859  59.362676    284  1.0      11139  39.221831   
2  Accessories 2016-08-03  19030  55.806452    341  1.0      12231  35.868035   
3  Accessories 2016-08-04  15020  52.334495    287  1.0      10648  37.101045   
4  Accessories 2016-08-05  19344  58.618182    330  1.0      13735  41.621212   

  first_visits           time_on_site              transactions  
           sum      mean          sum         mean          sum  
0           99  0.370787       261313   978.700375           62  
1          121  0.426056       322033  1133.919014           30  
2          176  0.516129       348664  1022.475073            0  
3          205  0.714286       423308  1474.940767            4  
4          210  0.636364       419308  1270.630303           85

In [24]:
def rename_column(tup):
    return '_'.join(filter(None, tup))

df_totals.columns = [rename_column(tup) for tup in df_totals.columns]

print(df_totals.info())
df_totals.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1805 entries, 0 to 1804
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   category           1805 non-null   object        
 1   date               1805 non-null   datetime64[ns]
 2   hits_sum           1805 non-null   Int64         
 3   hits_mean          1805 non-null   Float64       
 4   visits_sum         1805 non-null   Int64         
 5   visits_mean        1805 non-null   Float64       
 6   page_views_sum     1805 non-null   Int64         
 7   page_views_mean    1805 non-null   Float64       
 8   first_visits_sum   1805 non-null   Int64         
 9   first_visits_mean  1805 non-null   Float64       
 10  time_on_site_sum   1805 non-null   Int64         
 11  time_on_site_mean  1805 non-null   Float64       
 12  transactions_sum   1805 non-null   Int64         
dtypes: Float64(5), Int64(6), datetime64[ns](1), object(1)
memory us

,category,date,hits_sum,hits_mean,visits_sum,visits_mean,page_views_sum,page_views_mean,first_visits_sum,first_visits_mean,time_on_site_sum,time_on_site_mean,transactions_sum
0,Accessories,2016-08-01,13699,51.307116,267,1.0,9083,34.018727,99,0.370787,261313,978.700375,62
1,Accessories,2016-08-02,16859,59.362676,284,1.0,11139,39.221831,121,0.426056,322033,1133.919014,30
2,Accessories,2016-08-03,19030,55.806452,341,1.0,12231,35.868035,176,0.516129,348664,1022.475073,0
3,Accessories,2016-08-04,15020,52.334495,287,1.0,10648,37.101045,205,0.714286,423308,1474.940767,4
4,Accessories,2016-08-05,19344,58.618182,330,1.0,13735,41.621212,210,0.636364,419308,1270.630303,85


In [52]:
query = """
SELECT
  CASE 
    WHEN hits.eCommerceAction.action_type = '1' THEN 'Click through of product lists'
    WHEN hits.eCommerceAction.action_type = '2' THEN 'Product detail views'
    WHEN hits.eCommerceAction.action_type = '5' THEN 'Check out'
    WHEN hits.eCommerceAction.action_type = '6' THEN 'Completed purchase'
  END as action,
  products.v2ProductCategory as category,
  date,
  trafficSource.medium as medium,
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*` h,
    UNNEST(h.hits) hits,
    UNNEST(hits.product) products
WHERE 
  (_TABLE_SUFFIX BETWEEN '20160801' AND '20170801')
  AND
    (
      hits.eCommerceAction.action_type != '0'
      AND
      hits.eCommerceAction.action_type != '3'
      AND
      hits.eCommerceAction.action_type != '4'
    )
  AND
    (geoNetwork.country = 'United States')
  AND
  (
  products.v2ProductCategory LIKE '%Accessories%'
  OR products.v2ProductCategory LIKE '%Electronics%'
  OR products.v2ProductCategory LIKE '%Office%'
  OR products.v2ProductCategory LIKE "%Men's%"
  OR products.v2ProductCategory LIKE "%Women's%"
  OR products.v2ProductCategory LIKE "%Kid's%"
  OR products.v2ProductCategory LIKE "%Apparel%"
  OR products.v2ProductCategory LIKE '%Shop by Brand%'
  )
"""

df_hits_traffic = client.query(query).to_dataframe()
print(df_hits_traffic.info())
df_hits_traffic.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 523380 entries, 0 to 523379
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   action    523380 non-null  object
 1   category  523380 non-null  object
 2   date      523380 non-null  object
 3   medium    523380 non-null  object
dtypes: object(4)
memory usage: 16.0+ MB
None


,action,category,date,medium
0,Click through of product lists,Home/Apparel/Kid's/Kids-Youth/,20170129,(none)
1,Click through of product lists,Home/Shop by Brand/YouTube/,20170129,referral
2,Product detail views,Home/Shop by Brand/YouTube/,20170129,referral
3,Product detail views,Home/Shop by Brand/YouTube/,20170129,referral
4,Product detail views,Home/Apparel/Men's/Men's-T-Shirts/,20170129,(none)


In [38]:
df_hits_traffic['date'] = pd.to_datetime(df_hits_traffic['date'])
df_hits_traffic['category'] = df_hits_traffic['category'].str.split('/').apply(categorized)

print(df_hits_traffic.info())
df_hits_traffic.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 523380 entries, 0 to 523379
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   action    523380 non-null  object        
 1   category  523380 non-null  object        
 2   date      523380 non-null  datetime64[ns]
 3   medium    523380 non-null  object        
dtypes: datetime64[ns](1), object(3)
memory usage: 16.0+ MB
None


,action,category,date,medium
0,Product detail views,Apparel,2017-04-09,organic
1,Click through of product lists,Apparel,2017-04-09,organic
2,Click through of product lists,Apparel,2017-04-09,organic
3,Product detail views,Apparel,2017-04-09,organic
4,Click through of product lists,Accessories,2017-04-09,organic


In [39]:
df_ohe = pd.get_dummies(df_hits_traffic[['medium', 'action']], dtype=int)\
            .drop(['medium_(not set)', 'medium_(none)'], axis=1)
df_hits_traffic = pd.concat([df_hits_traffic.drop(['medium', 'action'], axis=1),
                            df_ohe], axis=1)

# def modify_columns(array):
#     temp = []
#     for d in array:
#         temp.append(d.replace('(', '').replace(')', '').replace(' ', '_'))
#     return temp


# df_hits_traffic.columns = modify_columns(np.array(df_hits_traffic.columns))

print(df_hits_traffic.info())
df_hits_traffic.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 523380 entries, 0 to 523379
Data columns (total 11 columns):
 #   Column                                 Non-Null Count   Dtype         
---  ------                                 --------------   -----         
 0   category                               523380 non-null  object        
 1   date                                   523380 non-null  datetime64[ns]
 2   medium_affiliate                       523380 non-null  int32         
 3   medium_cpc                             523380 non-null  int32         
 4   medium_cpm                             523380 non-null  int32         
 5   medium_organic                         523380 non-null  int32         
 6   medium_referral                        523380 non-null  int32         
 7   action_Check out                       523380 non-null  int32         
 8   action_Click through of product lists  523380 non-null  int32         
 9   action_Completed purchase              523380 no

,category,date,medium_affiliate,medium_cpc,medium_cpm,medium_organic,medium_referral,action_Check out,action_Click through of product lists,action_Completed purchase,action_Product detail views
0,Apparel,2017-04-09,0,0,0,1,0,0,0,0,1
1,Apparel,2017-04-09,0,0,0,1,0,0,1,0,0
2,Apparel,2017-04-09,0,0,0,1,0,0,1,0,0
3,Apparel,2017-04-09,0,0,0,1,0,0,0,0,1
4,Accessories,2017-04-09,0,0,0,1,0,0,1,0,0


In [40]:
df_hits_traffic = df_hits_traffic.groupby(['category', 'date']).sum().reset_index()

print(df_hits_traffic.info())
df_hits_traffic.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1805 entries, 0 to 1804
Data columns (total 11 columns):
 #   Column                                 Non-Null Count  Dtype         
---  ------                                 --------------  -----         
 0   category                               1805 non-null   object        
 1   date                                   1805 non-null   datetime64[ns]
 2   medium_affiliate                       1805 non-null   int32         
 3   medium_cpc                             1805 non-null   int32         
 4   medium_cpm                             1805 non-null   int32         
 5   medium_organic                         1805 non-null   int32         
 6   medium_referral                        1805 non-null   int32         
 7   action_Check out                       1805 non-null   int32         
 8   action_Click through of product lists  1805 non-null   int32         
 9   action_Completed purchase              1805 non-null   int32   

,category,date,medium_affiliate,medium_cpc,medium_cpm,medium_organic,medium_referral,action_Check out,action_Click through of product lists,action_Completed purchase,action_Product detail views
0,Accessories,2016-08-01,0,3,3,57,14,0,146,0,121
1,Accessories,2016-08-02,0,14,0,102,2,0,151,0,133
2,Accessories,2016-08-03,0,15,2,140,6,0,179,0,162
3,Accessories,2016-08-04,8,2,0,93,13,0,152,0,135
4,Accessories,2016-08-05,7,20,0,192,0,0,180,0,150


In [41]:
df = pd.merge(df_totals, df_hits_traffic, how='outer', on=['category', 'date'])
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1805 entries, 0 to 1804
Data columns (total 22 columns):
 #   Column                                 Non-Null Count  Dtype         
---  ------                                 --------------  -----         
 0   category                               1805 non-null   object        
 1   date                                   1805 non-null   datetime64[ns]
 2   hits_sum                               1805 non-null   Int64         
 3   hits_mean                              1805 non-null   Float64       
 4   visits_sum                             1805 non-null   Int64         
 5   visits_mean                            1805 non-null   Float64       
 6   page_views_sum                         1805 non-null   Int64         
 7   page_views_mean                        1805 non-null   Float64       
 8   first_visits_sum                       1805 non-null   Int64         
 9   first_visits_mean                      1805 non-null   Float64 

,category,date,hits_sum,hits_mean,visits_sum,visits_mean,page_views_sum,page_views_mean,first_visits_sum,first_visits_mean,...,transactions_sum,medium_affiliate,medium_cpc,medium_cpm,medium_organic,medium_referral,action_Check out,action_Click through of product lists,action_Completed purchase,action_Product detail views
0,Accessories,2016-08-01,13699,51.307116,267,1.0,9083,34.018727,99,0.370787,...,62,0,3,3,57,14,0,146,0,121
1,Accessories,2016-08-02,16859,59.362676,284,1.0,11139,39.221831,121,0.426056,...,30,0,14,0,102,2,0,151,0,133
2,Accessories,2016-08-03,19030,55.806452,341,1.0,12231,35.868035,176,0.516129,...,0,0,15,2,140,6,0,179,0,162
3,Accessories,2016-08-04,15020,52.334495,287,1.0,10648,37.101045,205,0.714286,...,4,8,2,0,93,13,0,152,0,135
4,Accessories,2016-08-05,19344,58.618182,330,1.0,13735,41.621212,210,0.636364,...,85,7,20,0,192,0,0,180,0,150


In [48]:
df.to_csv('../result/data_modeling.csv', index=False)
pd.DataFrame(df['category'].unique(), columns=['category']).to_csv('../result/category.csv', index_label='id')